# Part4-可视化好助手Tensorboard2--可视化训练过程

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


1. tf.histogram_summary()方法,用来绘制图片, 第一个参数是图表的名称, 第二个参数是图表要记录的变量
2. tf.histogram_summary()即tf.summary.histogram()
3. tf.merge_all_summaries() 将所有的summaries合并在一起

## 1. 在layer中为Weights,biases设置为图表

- 在add_layer()中添加参数n_layer,用来标识层数, 并且用变量 layer_name 代表其每层的名名称
- tf.summary.histogram()方法,用来绘制图片, 第一个参数是图表的名称, 第二个参数是图表要记录的变量

#### 定义add_layer()

In [7]:
def add_layer(inputs, in_size, out_size, n_layer, 
              activation_function=None):
    layer_name = 'layer%s'%n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]),
                                 name='W')
            #绘制weights的图表
            tf.summary.histogram(layer_name+'/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size])+0.1, name='b')
            #绘制biases的图表
            tf.summary.histogram(layer_name+'/biases', biases)
        with tf.name_scope('Wx_plus_b'):    
            Wx_plus_b = tf.add(tf.matmul(inputs,Weights), biases)
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
         #绘制outputs的图表
        tf.summary.histogram(layer_name+'/outputs', outputs)
        return outputs    

In [5]:
x_data = np.linspace(-1,1,300)[:, np.newaxis]
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32,[None,1], name='x_input')
    ys = tf.placeholder(tf.float32,[None,1], name='y_input')

#### 定义神经网络

In [9]:
l1 = add_layer(xs,1,10, n_layer=1, activation_function=tf.nn.relu)

prediction = add_layer(l1,10,1,n_layer=2, activation_function=None)

# 计算损失
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction), 
                                        reduction_indices=[1]))
    #  定义损失值得图表
    tf.summary.scalar('loss',loss)

# 训练
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    train_step = optimizer.minimize(loss)
#     tf.summary.scalar('train', train_step)

#### 所有训练图合并

In [14]:
sess = tf.Session()
merged = tf.summary.merge_all()#所有训练图合并
# 图表保存到文件夹
writer = tf.summary.FileWriter('logs/', sess.graph)

#### 变量初始化并激活

In [12]:
init = tf.global_variables_initializer()
sess.run(init)

#### 训练

In [13]:
for i in range(1000):
    sess.run(train_step, feed_dict={xs:x_data, ys:y_data})
    if i%50 == 0:
        result = sess.run(merged,feed_dict={xs:x_data, ys:y_data})
        writer.add_summary(result, i)